# Tau to Nissl registration using color deconvolution


In [ ]:
import subprocess
import sys

from deconvolution import Deconvolution
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import SimpleITK as sitk
import skimage
from sklearn.ensemble import RandomForestClassifier


# https://github.com/pyushkevich/histoannot.git
# git clone the GitHub repository and add the path to the sys.path
# This makes sure you are using the latest version of the code
sys.path.append("/Users/cathalye/Packages/histoannot/")
import phas.client.api as phas
from picsl_c3d import Convert3D
from picsl_c3d import Convert2D
from picsl_greedy import Greedy3D
from picsl_greedy import Greedy2D

## Connect to server
User input needed - `PHAS_URL`, `PRIVATE_KEY`

In [ ]:
# To get the API key you need to go to https://histo.itksnap.org/auth/api/generate_key
PHAS_URL='https://histo.itksnap.org'
PRIVATE_KEY = '/Users/cathalye/.private/histoitk_api_key.json'

conn = phas.Client(PHAS_URL, PRIVATE_KEY)
proj_listing = pd.DataFrame(conn.project_listing())
proj_listing

In [ ]:
task_listing = pd.DataFrame(conn.task_listing('mtl'))
task_listing

### Choose task ID
User input needed - `TASK_ID` (choose the id number from `task_listing` dataframe above)

In [ ]:
TASK_ID = 39

# Create a task object to pass to Slide class for downloading thumbnails
task = phas.Task(conn, TASK_ID)

# Create a SamplingROITask object for downloading the ROIs as segmentations
samplingROI_task = phas.SamplingROITask(conn, TASK_ID)

## Download source and target slides
User input needed - `SRC_ID`, `TRG_ID` (choose the slide number from the server UI)

In [ ]:
REF_ID = 41867 # Tau fixed image
MOV_ID = 43788 # Nissl image with annotations

reference_slide = phas.Slide(task, REF_ID)
reference_slide.thumbnail_nifti_image('reference_thumbnail.nii.gz')

moving_slide = phas.Slide(task, MOV_ID)
moving_slide.thumbnail_nifti_image('moving_thumbnail.nii.gz')
samplingROI_task.slide_sampling_roi_nifti_image(MOV_ID, 'moving_sampling_roi.nii.gz')

# Registration

In [ ]:
# Step 1 - Color deconvolution to get similar looking images

reference_thumbnail = sitk.ReadImage('reference_thumbnail.nii.gz')
moving_thumbnail = sitk.ReadImage('moving_thumbnail.nii.gz')

reference_im = Image.fromarray(sitk.GetArrayFromImage(reference_thumbnail)[0,:,:,:])
decimg = Deconvolution(image=reference_im, sample_density=6)
decimg.complete_basis()
decimg.resolve_dependencies()
reference_layer1, reference_layer2 = decimg.out_images(mode=[1, 2])


In [ ]:
sitk.WriteImage(sitk.GetImageFromArray(reference_layer2), "referece_layer2.nii.gz")

moving_im = Image.fromarray(sitk.GetArrayFromImage(moving_thumbnail)[0,:,:,:])
decimg = Deconvolution(image=moving_im, sample_density=6)
decimg.complete_basis()
decimg.resolve_dependencies()
moving_layer1, moving_layer2 = decimg.out_images(mode=[1, 2])


In [ ]:
print(np.array(moving_layer2).shape)
plt.imshow(np.array(moving_layer2)[:,:,0])

In [ ]:
# Step 2 - Get binanry mask of the reference image

# reference_layer2 = skimage.io.imread('reference_layer2.png')
reference_layer2_gray = skimage.color.rgb2gray(reference_layer2)*255

X, Y = reference_layer2_gray.shape
reference_layer2_gray[0:25, :] = 255
reference_layer2_gray[X-25:X, :] = 255
reference_layer2_gray[:, 0:25] = 255
reference_layer2_gray[:, Y-25:Y] = 255

thresh = skimage.filters.threshold_otsu(reference_layer2_gray)
reference_binary_mask = reference_layer2_gray < thresh

skimage.io.imsave('reference_binary_mask.png', reference_binary_mask)


In [ ]:
# Step 3 - Divide the mask into chunks using image graph cut
# For simplicity, we will define a fixed number of chunks

N_CHUNKS = 10

subprocess.run('image_graph_cut -u 1.2 -n 100 -c 4 0.1 reference_binary_mask.png reference_multi_chunk.nii.gz {}'.format(N_CHUNKS), shell=True)


In [ ]:
# Step 4 - Perform global rigid registration

greedy = Greedy2D()

# greedy.execute('-i reference_layer2.png moving_layer2.png '
#                '-a -dof 6 '
#                '-gm reference_binary_mask.png '
#                '-ia-image-centers '
#                '-m WNCC 4x4 -bg NaN -wncc-mask-dilate '
#                '-n 200x200x40x0x0 -search 20000 any 10 '
#                '-o output_global_rigid.mat',
#                )

subprocess.run('greedy -d 2 -a -dof 6 -i referece_layer2.nii.gz moving_layer2.nii.gz -gm reference_binary_mask.nii.gz -ia-image-centers -m WNCC 4x4 -wncc-mask-dilate -n 200x200x40x0x0 -search 20000 any 10 -o output_global_rigid.mat', shell=True)

In [ ]:
# Step 5 - Piecewise rigid registration

output_piecewise_rigid = "output_piecewise_rigid_%02d.mat"
subprocess.run('multi_chunk_greedy -d 2 -a -dof 6 -i reference_layer2.nii.gz moving_layer2.nii.gz -cm reference_multi_chunk.nii.gz -ia output_global_rigid.mat -m WNCC 4x4 -wncc-mask-dilate -n 600x600x200x0 -search 10000 10 5 -wreg 0.05 -o {}'.format(output_piecewise_rigid), shell=True)


In [ ]:
# Step 6 - Piecewise deformable registration

output_piecewise_deformable = "output_piecewise_deformable_%02d.nii.gz"
subprocess.run('multi_chunk_greedy -d 2 -i reference_layer2.nii.gz moving_layer2.nii.gz -cm reference_multi_chunk.nii.gz -it {} -m WNCC 4x4 -wncc-mask-dilate -n 400x200x100x20 -sv -s 0.6mm 0.1mm -e 0.25 -o {}'.format(output_piecewise_rigid, output_piecewise_deformable), shell=True)

In [ ]:
# Step 7 - Apply the piecewise rigid and deformable registration to the moving image

subprocess.run('multi_chunk_greedy -d 2 -rf reference_thumbnail.nii.gz -cm reference_multi_chunk.nii.gz -r {} {} -rb 255 -rm moving_thumbnail.nii.gz moving_deform.nii.gz'.format(output_piecewise_deformable, output_piecewise_rigid), shell=True)